In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import sys
RunPath = os.getcwd() + "/"
CrysDatPath = "../CrysDat_FCC/" #"/home/sohamc2/HEA_FCC/MDMC/CrysDat_FCC/"
DataPath = "../MD_KMC_single/" #"/home/sohamc2/HEA_FCC/MDMC/ML_runs/DataSets/"
ModulePath = "../Symm_Network/" #"/home/sohamc2/VKMC/SymNetworkRuns/CE_Symmetry/Symm_Network/"

In [3]:
import numpy as np
import h5py
import torch as pt
from tqdm import tqdm
import pickle
from GCNetRun import Load_Data, makeComputeData, makeDataTensors, Load_crysDats, SpecBatchOuts, vacBatchOuts

In [4]:
T = 1073
state1List, state2List, dispList, rateList, AllJumpRates_st1,\
AllJumpRates_st2, avgDisps_st1, avgDisps_st2 = Load_Data(DataPath + "singleStep_{}_AllRates.h5".format(T))

found permuation


In [5]:
GpermNNIdx, NNsiteList, JumpNewSites, dxJumps = Load_crysDats(1, CrysDatPath)

z = dxJumps.shape[0]

RtoSiteInd = np.load(CrysDatPath + "RtoSiteInd.npy")
SiteIndToR = np.load(CrysDatPath + "SiteIndtoR.npy")

with open(CrysDatPath + "supercellFCC.pkl", "rb") as fl:
    superFCC = pickle.load(fl)
N_units = superFCC.superlatt[0,0]
N_units

8

# First test with single step dataset

In [6]:
specCheck = 5
specsToTrain = [specCheck]
VacSpec = 0
N_check = 200
AllJumps = False

State1_occs, State2_occs, rates, disps, OnSites_state1, OnSites_state2, sp_ch =\
makeComputeData(state1List, state2List, dispList, specsToTrain, VacSpec, rateList,
                AllJumpRates_st1, AllJumpRates_st2, avgDisps_st1, avgDisps_st2, JumpNewSites,
                dxJumps, NNsiteList, N_check, AllJumps=AllJumps, mode="train")

Building Occupancy Tensors for species : [5]
No. of jumps : 200


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1625.85it/s]


## Test Occupancies

In [7]:
State1_occs.shape

(200, 5, 512)

In [8]:
for samp in tqdm(range(N_check), position=0, leave=True):
    for site in range(state1List.shape[1]):
        if site == 0:
            assert np.all(State1_occs[samp, :, site] == 0)
        else:
            spec1 = state1List[samp, site]
            assert State1_occs[samp, spec1 - 1, site] == 1
            assert np.sum(State1_occs[samp, :, site]) == 1
            
            spec2 = state2List[samp, site]
            assert State2_occs[samp, spec2 - 1, site] == 1
            assert np.sum(State2_occs[samp, :, site]) == 1
        
    # check the displacements
    jSelect = None
    count = 0
    for jInd in range(JumpNewSites.shape[0]):
        state2_try = state1List[samp][JumpNewSites[jInd]]
        if np.all(state2_try == state2List[samp]):
            jSelect = jInd
            count += 1
    assert count == 1
        
    NNR = np.dot(np.linalg.inv(superFCC.crys.lattice), dxJumps[jSelect]).astype(int)
    assert np.allclose(np.dot(superFCC.crys.lattice, NNR), dxJumps[jSelect])
    NNRSite = superFCC.index(NNR, (0, 0))[0]
    
    NNsiteVac = NNsiteList[jSelect + 1, 0]
    assert np.all(NNRSite == NNsiteVac)
    
    spec = state1List[samp, NNsiteVac]
    assert np.allclose(disps[samp, 0], 3.59*dxJumps[jSelect])
    if spec == specCheck:
        assert np.allclose(disps[samp, 1], -3.59*dxJumps[jSelect])
    else:
        assert np.allclose(disps[samp, 1], 0.0)
    
    # check the rate
    assert np.math.isclose(rates[samp], rateList[samp])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 50.56it/s]


## Check the batched calculation of y vectors

### Check without jump channels - tests shifted to new notebook for those

In [9]:
state1Data, state2Data, dispData, rateData, On_st1, On_st2 =\
makeDataTensors(State1_occs, State2_occs, rates, disps, OnSites_state1, OnSites_state2,
                specsToTrain, VacSpec, sp_ch, Ndim=3)

Training Species : [5]


In [10]:
On_st1.shape, dispData.shape

(torch.Size([200, 512]), torch.Size([200, 3]))

In [11]:
pt.dot(rateData, pt.norm(dispData, dim=1)**2)/(6.0 * dispData.shape[0])

tensor(0.0386, dtype=torch.float64)

In [14]:
batchStart = 0
batchEnd = batchStart + N_check
y = pt.rand(N_check, 1, 3, state1List.shape[1]).double()

jProbs_st1 = None
jProbs_st2 = None
NNsvac_st1 = None
NNsvac_st2 = None

y1, y2 = SpecBatchOuts(y, y, On_st1, On_st2, jProbs_st1, jProbs_st2,
                       NNsvac_st1, NNsvac_st2, False, False)

for sampInd in tqdm(range(batchStart, batchEnd), position=0, leave=True):
    y1_samp = pt.zeros(3).double()
    y2_samp = pt.zeros(3).double()
    for site in range(On_st1.shape[1]):
        occs1 = State1_occs[sampInd, :, site]
        occs2 = State2_occs[sampInd, :, site]
        if occs1[specCheck-1] == 1:
            y1_samp += y[sampInd - batchStart, 0, :, site]
        if occs2[specCheck-1] == 1:
            y2_samp += y[sampInd - batchStart, 0, :, site]
    
    assert pt.allclose(y1[sampInd - batchStart], y1_samp)
    assert pt.allclose(y2[sampInd - batchStart], y2_samp)

100%|████████████████████████████████████████| 200/200 [00:01<00:00, 123.35it/s]


# now let's try with all jumps

In [15]:
specCheck = 5
specsToTrain = [specCheck]
VacSpec = 0
AllJumps = True
State1_occs, State2_occs, rates, disps, OnSites_state1, OnSites_state2, sp_ch =\
makeComputeData(state1List, state2List, dispList, specsToTrain, VacSpec, rateList,
                AllJumpRates_st1, AllJumpRates_st2, avgDisps_st1, avgDisps_st2, JumpNewSites,
                dxJumps, NNsiteList, N_check, AllJumps=AllJumps)

assert State1_occs.shape[0] == State2_occs.shape[0] == rates.shape[0] == disps.shape[0] == N_check * z
assert OnSites_state1.shape[0] == OnSites_state2.shape[0] == N_check * z

state1Data, state2Data, dispData, rateData, On_st1, On_st2 =\
makeDataTensors(State1_occs, State2_occs, rates, disps, OnSites_state1, OnSites_state2,
                specsToTrain, VacSpec, sp_ch, Ndim=3)

assert state1Data.shape[0] == state2Data.shape[0] == rateData.shape[0] == dispData.shape[0] == N_check * z
assert On_st1.shape[0] == On_st2.shape[0] == N_check * z

Building Occupancy Tensors for species : [5]
No. of jumps : 2400


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 134.15it/s]

Training Species : [5]


## Test Occupancies

In [16]:
for stateInd in tqdm(range(N_check), position=0, leave=True):
    state1 = state1List[stateInd]
    for jInd in range(z):
        state2 = state1.copy()
        NNsiteVac = NNsiteList[jInd + 1, 0]
        _, RsiteVac = superFCC.ciR(NNsiteVac)
        state2[0] = state2[NNsiteVac]
        state2[NNsiteVac] = 0
        
        for site in range(state1List.shape[1]):
            _, Rsite = superFCC.ciR(site)
            RsiteNew = (Rsite + RsiteVac) % N_units
            siteNew = RtoSiteInd[RsiteNew[0], RsiteNew[1], RsiteNew[2]]
            spec1 = state1[site]
            spec2 = state2[siteNew]
            
            if site == 0:
                assert spec2 == 0
                assert np.all(State1_occs[stateInd * dxJumps.shape[0] + jInd, :, site] == 0)
                assert np.all(State2_occs[stateInd * dxJumps.shape[0] + jInd, :, site] == 0)
            
            else:
                assert spec2 != 0
                assert State1_occs[stateInd * dxJumps.shape[0] + jInd, spec1-1, site] == 1
                assert np.sum(State1_occs[stateInd * dxJumps.shape[0] + jInd, :, site]) == 1

                assert State2_occs[stateInd * dxJumps.shape[0] + jInd, spec2-1, site] == 1
                assert np.sum(State2_occs[stateInd * dxJumps.shape[0] + jInd, :, site]) == 1
        
        # check the displacements
        NNR = np.dot(np.linalg.inv(superFCC.crys.lattice), dxJumps[jInd]).astype(int) % N_units
        NNsiteVac = NNsiteList[jInd + 1, 0]
        assert np.all(NNR == SiteIndToR[NNsiteVac])
        spec = state1[NNsiteVac]
        assert np.allclose(disps[stateInd * dxJumps.shape[0] + jInd, 0], 3.59*dxJumps[jInd])
        if spec == specCheck:
            assert np.allclose(disps[stateInd * dxJumps.shape[0] + jInd, 1], -3.59*dxJumps[jInd])
            assert np.allclose(disps[stateInd * dxJumps.shape[0] + jInd, 1],
                               -3.59*pt.tensor(dxJumps[jInd], dtype=pt.double))
        
        # check the rate
        assert np.allclose(rateData[stateInd * dxJumps.shape[0] + jInd], AllJumpRates_st1[stateInd, jInd])

100%|█████████████████████████████████████████| 200/200 [01:09<00:00,  2.88it/s]


In [17]:
batchStart = 0
batchEnd = batchStart + N_check * z
y = pt.rand(N_check * z, 1, 3, state1List.shape[1]).double()

jProbs_st1 = None
jProbs_st2 = None
NNsvac_st1 = None
NNsvac_st2 = None

y1, y2 = SpecBatchOuts(y, y, On_st1, On_st2, jProbs_st1, jProbs_st2,
                       NNsvac_st1, NNsvac_st2, False, False)

for sampInd in tqdm(range(batchStart, batchEnd), position=0, leave=True):
    y1_samp = pt.zeros(3).double()
    y2_samp = pt.zeros(3).double()
    for site in range(On_st1.shape[1]):
        occs1 = State1_occs[sampInd, :, site]
        occs2 = State2_occs[sampInd, :, site]
        
        if occs1[specCheck-1] == 1:
            y1_samp += y[sampInd - batchStart, 0, :, site]
        if occs2[specCheck-1] == 1:
            y2_samp += y[sampInd - batchStart, 0, :, site]
    
    assert pt.allclose(y1[sampInd - batchStart], y1_samp)
    assert pt.allclose(y2[sampInd - batchStart], y2_samp)

100%|██████████████████████████████████████| 2400/2400 [00:18<00:00, 130.96it/s]
